In [191]:
import json
import math
import pandas as pd

What we're aiming for is something like this

```json
{
  "E14999999":{
    "name": "Fake news",
    "code": "E14999999",
    "description": "This constituency is great!",
    "headlines": [
      {"value":1,"name":"People"},
      {"value":1,"name":"People"},
      {"value":1,"name":"People"},
      {"value":1,"name":"People"}
    ],
		"similar": { "People": 4 },
		"leastsimilar": { "People": 1000 },
		"neet": {
			"something": [34,56,24,19,25],
			"another": {}
		},
		"employment": {
			
		}
	}
}
```

In [192]:
descriptions=pd.read_csv('../../data/reference/constituency-descriptions.csv', index_col='PCON22CD').fillna('')

In [193]:
pcons=pd.read_json('../../src/_data/areas/reference/pcon.json')

In [194]:
summary=pd.merge(pcons, descriptions, how='left', left_on='PCON22CD', right_index=True).set_index('PCON22CD')

summary.rename(columns={ 'PCON22NM': 'name'}, inplace=True)
summary.rename(columns=lambda x: x.lower(), inplace=True)

In [195]:
summary = summary.to_dict(orient='index')

In [196]:
headlines = pd.read_csv('../../data/area/pcon/headlines.csv', index_col='PCON22CD')
econ_inactive = pd.read_csv('../../data/area/pcon/econ_inactive_16-24_last_3_years.csv', index_col='PCON22CD')
unemployment = pd.read_csv('../../data/area/pcon/youth_unem_16-24_last_3_years.csv', index_col='PCON22CD')
economic_activity_16_19 = pd.read_csv('../../data/area/pcon/econ_active_16-19_last_3_years.csv', index_col='PCON22CD')
economic_activity_20_24 = pd.read_csv('../../data/area/pcon/econ_active_20-24_last_3_years.csv', index_col='PCON22CD')

In [197]:
for s in summary:
    suffix = headlines.loc[s, :].Suffix
    summary[s]['headlines'] = [
        { 'h': headline_name, 'v': headline_value, 'suffix': suffix }
        for headline_name, headline_value
        in headlines.loc[s, :].drop(labels="Suffix").items()
        ]
    try: 
        summary[s]['economic_inactivity'] = [
            { 'year': date_name, 'value': value }
            for date_name, value
            in econ_inactive.loc[s,:].items()
        ]
        # summary[s]['economic_activity_16_19'] = [
        #     { 'year': date_name, 'value': value }
        #     for date_name, value
        #     in economic_activity_16_19.loc[s,:].items()
        # ]
        # summary[s]['economic_activity_20_24'] = [
        #     { 'year': date_name, 'value': value }
        #     for date_name, value
        #     in economic_activity_20_24.loc[s,:].items()
        # ]
    except: 
        print(f'No data found for constituency: {s}')
    
    '''Do this for all other things...'''

In [198]:
with open('../../src/generated/areas/constituency/_data/summary.json', 'w') as f:
    json.dump(summary, f, indent=2)